In [1]:
import os
import openai
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import warnings
import re

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def url_to_string(url):
    warnings.filterwarnings('ignore', message='Unverified HTTPS request')
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()

def split_text(text, chunk_size=1000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        if end < len(text) and text[end] not in '.?!':
            while end < len(text) and text[end] not in '.?!':
                end += 1
        chunks.append(text[start:end + 1].strip())
        start = end + 1
    return chunks

def openai_edit(text, i = "Rewrite it in narrative style") :
    response = openai.Edit.create(
      model = "text-davinci-edit-001",
      input = text,
      instruction = i
    )   
    return response['choices'][0]['text']

def openai_complete(text, i = "in a narrative style rewrite text below") :
    response = openai.Completion.create(
        model = "text-davinci-003",
        prompt = i+"\n"+ text +"",
        temperature = 0.5,
        max_tokens = 2048,
        top_p = 1,
        frequency_penalty = 0.2,
    )       
    return response['choices'][-1]['text']

def rewrite_text(chunks, func):
    rewritten_text = ""
    for chunk in chunks:
        chunk_rewritten_text = func(chunk)
        rewritten_text += chunk_rewritten_text
    return rewritten_text
    
text = url_to_string('http://socalnews.com/la-story-3.html')
text_chunks = split_text(text)

small_text_example = "The Burbank-based Walt Disney Co. will slash 7,000 jobs as part of a $5.5 billion cost-cutting effort, CEO Bob Iger announced Wednesday.The possibility of layoffs at Disney has been rumored for weeks, with Iger regaining his footing in the CEO’s office following the surprise ouster of CEO Bob Chapek in November. Iger broke the news during an earnings call Wednesday afternoon, following the release of the company’s first-quarter earnings report. Iger said he is targeting $5.5 billion in cost savings “across the company.”"
print(text)


C:\Users\beaconproduction\sites\local\python\heywire\openai\venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Bass Inauguration, 3rd LdDate: 12-11-2022 8:50 PM - Word Count: 1283Bass Inauguration, 3rd Ld
   Karen Bass Inaugurated as Mayor, to Hold News Conference
   Eds: UPDATES with Bass to hold news conference, in 7th graf. Bass'
office can be reached at press@karenbass.com.
   By ERIC HE
   City News Service
   LOS ANGELES (CNS) - Karen Bass was sworn in as the 43rd mayor of Los
Angeles today in a historic inauguration at the Microsoft Theater, becoming the
first woman and second Black person to lead the city.
   Bass was sworn in by Vice President Kamala Harris, a former California
senator and the first woman to serve as the nation's second-in-command.
   Nearly every major city official, along with Gov. Gavin Newsom,
attended the ceremony.
   ``Making history with each of you today is a monumental moment in my
life and for Los Angeles,'' Bass said in her speech.
   Bass addressed what she described as an ``inflection point in our
history,'' with issues including ``the pandemic, the rapidl

In [47]:
# new_text = rewrite_text(text_chunks, openai_edit)
edit_model = openai_edit(small_text_example)
print("Example #1\nModel: Edit\n\n" +edit_model)

Example #1
Model: Edit

The Burbank-based Walt Disney Co. announced Wednesday it will slash 7,000 jobs as part of a $5.5 billion cost-cutting effort.
The possibility of layoffs had been rumored for weeks, with CEO Bob Iger regaining his footing in the CEO’s office following the surprise ouster of former CEO Bob Chapek in November.
Iger announced this during an earnings call Wednesday, following the release of the company’s first-quarter earnings report.
Iger said he is targeting $5.5 billion in cost savings “across the company.”



In [48]:
edit_model = openai_edit(small_text_example)
print("Example #2\nModel: Edit\n\n" +edit_model)

Example #2
Model: Edit

Earlier today, the Burbank-based Walt Disney Co. announced a plan to slash 7,000 jobs as part of a $5.5 billion cost-cutting effort.
CEO Bob Iger announced the plan during the company’s first-quarter earnings call on Wednesday.
The possibility of layoffs at Disney has been rumored for weeks, with Iger regaining his footing in the CEO’s office following the surprise ouster of CEO Bob Chapek in November.
Iger said he is targeting $5.5 billion in cost savings “across the company.”



In [56]:
complete_model = openai_complete(small_text_example)
print("Example #1\nModel: Complete" +complete_model)

Example #1
Model: Complete

On Wednesday, Bob Iger, CEO of the Burbank-based Walt Disney Company, confirmed weeks of speculation that the company would be cutting 7,000 jobs as part of a $5.5 billion cost-cutting effort. Iger made the announcement during an earnings call following the release of the company’s first-quarter earnings report. He stated that these reductions will be spread “across the company” and are necessary in order to reach their goal of $5.5 billion in cost savings. This news comes shortly after Iger took back his role as CEO following the unexpected departure of Bob Chapek in November.


In [55]:
complete_model = openai_complete(small_text_example)
print("Example #2\nModel: Complete" +complete_model)

Example #2
Model: Complete

Bob Iger, CEO of the Burbank-based Walt Disney Co., announced on Wednesday that the company will lay off 7,000 employees in a $5.5 billion cost-cutting effort. This news follows weeks of speculation and rumors after Bob Chapek was unexpectedly removed from the CEO's office in November. Iger made the announcement during an earnings call after the release of the company's first-quarter earnings report. He stated that the company is aiming for $5.5 billion in savings "across the company."


In [69]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
# import spacy
import en_core_web_sm
from pprint import pprint

def text_to_ents(text):
    nlp = en_core_web_sm.load()
    doc = nlp(text)
    unique_names = set([ent.text.lower() for ent in doc.ents])
    unique_ents = []
    for n in unique_names:
        appended = False
        for X in doc.ents:
            if X.text.lower() in n and appended == False and (X.text.lower(), X.label_) not in unique_ents:
                unique_ents.append((X.text.lower(), X.label_))
                appended = True
    return unique_ents
#     return doc.ents

entities = text_to_ents(text)
print(len(entities))
pprint([(X[0], X[1]) for X in entities])
# pprint([(X.text, X.label_) for X in entities])

88
[('69', 'DATE'),
 ('241-year', 'DATE'),
 ('african american', 'NORP'),
 ('2022', 'DATE'),
 ('city council', 'ORG'),
 ('fairfax', 'GPE'),
 ('daily', 'DATE'),
 ('five', 'CARDINAL'),
 ('bass', 'PERSON'),
 ('first', 'ORDINAL'),
 ('monday', 'DATE'),
 ('la', 'GPE'),
 ('six', 'CARDINAL'),
 ('toni atkins', 'PERSON'),
 ('city hall', 'FAC'),
 ("robert f. kennedy's", 'PERSON'),
 ('world war ii', 'EVENT'),
 ('los angeles', 'GPE'),
 ('the jim crow', 'ORG'),
 ('one', 'CARDINAL'),
 ('three', 'CARDINAL'),
 ('nearly a decade', 'DATE'),
 ('17,000', 'CARDINAL'),
 ('sunday', 'DATE'),
 ('l.a.', 'GPE'),
 ('14', 'DATE'),
 ('the congressional black caucus', 'ORG'),
 ('8:50 pm', 'TIME'),
 ('nov. 8', 'DATE'),
 ('chloe bailey', 'PERSON'),
 ('living for the city', 'WORK_OF_ART'),
 ('1968', 'DATE'),
 ('angelenos', 'PERSON'),
 ('stevie wonder', 'PERSON'),
 ('2013', 'DATE'),
 ('the u.s. house', 'ORG'),
 ('40,000', 'CARDINAL'),
 ('every month', 'DATE'),
 ("joe biden's", 'PERSON'),
 ('cns', 'ORG'),
 ('9 a.m.', 'TIM

In [62]:
from collections import Counter
labels = [x[1] for x in entities]
Counter(labels)

Counter({'DATE': 22,
         'NORP': 1,
         'ORG': 16,
         'GPE': 8,
         'CARDINAL': 8,
         'PERSON': 21,
         'ORDINAL': 4,
         'FAC': 2,
         'EVENT': 1,
         'TIME': 3,
         'WORK_OF_ART': 2})

In [68]:
items = [x.text for x in entities]
Counter(items).most_common(3)

[('Bass', 29), ('today', 7), ('first', 6)]

In [167]:
import wandb

run = wandb.init(project='HeywireAI', entity="heywire")

prediction_table = wandb.Table(columns=["original_story", "prompt", "usage", "completion"])

def generate_story(origin, entities, summary):

    mod = "text-davinci-003"
    prompt = "Generate a story that involves the following entities and origin example. Please make sure the story you generate is at least 500 words long."
    
#     completions = openai.Completion.create(
#         model=mod,
#         prompt=prompt,
#         max_tokens=1024
#     )

    entity_string = "\n".join([f"- {entity[0]}: {entity[1]}" for entity in entities])
    
    if summary:
        story_prompt = f"{prompt}\nOriginal example: {summary}\n Entities: {entity_string}\n\nBegin story:"
    else:
        story_prompt = f"{prompt}\n{entity_string}\n\nBegin story:"
    
    story_completions = openai.Completion.create(
        model = mod,
        prompt = story_prompt,
        max_tokens=int(4000 - (len(story_prompt) / 3)),
    )
        
    print(story_completions["usage"])
    
    prediction_table.add_data(origin, story_prompt, story_completions['usage'], story_completions['choices'][0]['text'])
    
    return story_completions['choices'][0]['text']

In [40]:
articles = [
    'https://socalnews.com/la-story-3.html',
    'https://socalnews.com/la-story-4.html',
    'https://socalnews.com/la-story-5.html',
    'https://socalnews.com/la-story-6.html',
    'https://socalnews.com/orange-story3.htm',
    'https://socalnews.com/orange-story4.htm'
]

def stories_run(stories):
    for story in stories:
        article_text = url_to_string(story)
        ents = text_to_ents(article_text)
        new_story = generate_story(article_text, ents)
        print(new_story)
    
    wandb.log({'predictions': prediction_table})
    wandb.finish()
    return

stories_run(articles)



Karen Bass was inaugurated as the 43rd mayor of Los Angeles today, at the Microsoft Theater. Bass, first elected to the Los Angeles City Council in 2007, and re-elected to the council in 2011 and 2015, was sworn in by California Senator Kamala Harris during a ceremony attended by Governor Gavin Newsom.

Bass addressed a crowd of 40,000 Angelenos gathered at City Hall, saying she was humbled to have been chosen as the city's first female African American mayor. She noted that the inauguration marked the 241-year anniversary of the first day of the City of Los Angeles.

The ceremony featured musical performances by Mark Gonzalez, Stevie Wonder, Chloe Bailey, and Las Cafeteras, as well as a poem by Amanda Gorman, the first person to hold the title of Youth Poet Laureate of the United States. Gorman read her poem at the inauguration of President Joe Biden's on Sunday.

Bass, who was elected mayor of Los Angeles on November 8, 2020, also noted that the inauguration was taking place after 



Robert Luna was appointed as the new sheriff of Los Angeles County on Saturday, replacing Jim McDonnell, who held the post for the past four years. Luna, who was previously the Long Beach police chief, was sworn in during a ceremony held at the Hall of Administration in downtown Los Angeles.

The ceremony was attended by more than 18,000 people, including Los Angeles Mayor Alex Villanueva and Long Beach Mayor Robert Garcia. Luna was also joined by Supervisor Holly Mitchell and other county officials.

In his speech, Luna thanked McDonnell for his service to the county and said he was honored to be the first sheriff in roughly a century to come from the Long Beach Police Department. He also thanked the people of Los Angeles County for their support in the Nov. 8 election, when he was elected to the post with more than 60 percent of the vote.

Luna then discussed his plans for the department, which include increasing the number of deputies to more than 18,000 and implementing new tan-a

In [70]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary

s = get_summary(text, 8)
text

"Bass Inauguration, 3rd LdDate: 12-11-2022 8:50 PM - Word Count: 1283Bass Inauguration, 3rd Ld\r\n\xa0\xa0 Karen Bass Inaugurated as Mayor, to Hold News Conference\r\n\xa0\xa0 Eds: UPDATES with Bass to hold news conference, in 7th graf. Bass'\r\noffice can be reached at press@karenbass.com.\r\n\xa0\xa0 By ERIC HE\r\n\xa0\xa0 City News Service\r\n\xa0\xa0 LOS ANGELES (CNS) - Karen Bass was sworn in as the 43rd mayor of Los\r\nAngeles today in a historic inauguration at the Microsoft Theater, becoming the\r\nfirst woman and second Black person to lead the city.\r\n\xa0\xa0 Bass was sworn in by Vice President Kamala Harris, a former California\r\nsenator and the first woman to serve as the nation's second-in-command.\r\n\xa0\xa0 Nearly every major city official, along with Gov. Gavin Newsom,\r\nattended the ceremony.\r\n\xa0\xa0 ``Making history with each of you today is a monumental moment in my\r\nlife and for Los Angeles,'' Bass said in her speech.\r\n\xa0\xa0 Bass addressed what she d

In [102]:
new_story_summary = generate_story(text, entities, s)
new_story_summary

1350.0


'\nKaren Bass was inaugurated as the 43rd mayor of Los Angeles on Sunday at the Microsoft Theater, making her the first woman and second African American to hold the office. Bass was sworn in by California\'s first and second female U.S. senators, Kamala Harris and Dianne Feinstein, and was joined by Gov. Gavin Newsom and other state and local officials. \n\nBass, who was elected in November with more than 40,000 votes, thanked Angelenos for their support. “It’s been nine days since the election and I’m already getting to work," she said, adding that she was already preparing for her first day in office on Monday at 9 a.m.\n\nBass, who previously served in the state legislature and the U.S. House, is the first mayor of Los Angeles in nearly a decade who is not Eric Garcetti. After being elected in 2013, Garcetti was reelected in 2017 and was term-limited from running again. \n\nBass won the election with a campaign focused on economic and racial justice, and on Sunday she promised to c

In [103]:
wandb.log({'predictions': prediction_table})
wandb.finish()
print(new_story_summary)


Karen Bass was inaugurated as the 43rd mayor of Los Angeles on Sunday at the Microsoft Theater, making her the first woman and second African American to hold the office. Bass was sworn in by California's first and second female U.S. senators, Kamala Harris and Dianne Feinstein, and was joined by Gov. Gavin Newsom and other state and local officials. 

Bass, who was elected in November with more than 40,000 votes, thanked Angelenos for their support. “It’s been nine days since the election and I’m already getting to work," she said, adding that she was already preparing for her first day in office on Monday at 9 a.m.

Bass, who previously served in the state legislature and the U.S. House, is the first mayor of Los Angeles in nearly a decade who is not Eric Garcetti. After being elected in 2013, Garcetti was reelected in 2017 and was term-limited from running again. 

Bass won the election with a campaign focused on economic and racial justice, and on Sunday she promised to continue t

In [72]:
story_1 = """Another Chinese Balloon Flew Over Latin America, China Confirms
The Chinese government said the balloon that Colombia spotted in its airspace on Friday was for civilian purposes and was being used for flight tests.
Give this article
A Chinese balloon floating off the coast of South Carolina before being shot down by an American fighter jet on Saturday.
A Chinese balloon floating off the coast of South Carolina before being shot down by an American fighter jet on Saturday.Credit...Chad Fish, via Associated Press
Emma Bubola
By Emma Bubola
Feb. 6, 2023
A Chinese balloon floated over Latin America and the Caribbean, the Chinese government confirmed on Monday, adding that it was for civilian purposes and was being used for flight tests.
“Affected by the weather and with limited self-steering capability, the airship deviated far from its planned course and entered into the airspace of Latin America and the Caribbean,” a spokeswoman for China’s foreign ministry, Mao Ning, said at a news conference on Monday.
On Friday morning, before the United States shot down a Chinese spy balloon that had spent the last week traversing the country, the Colombian National Air Defense System detected an object that had entered the northern part of the country’s airspace, the Colombian Air Force said in a statement.
Officials determined that the object had “characteristics similar to those of a balloon,” and that it was flying at an altitude of over 55,000 feet and moving at an average speed of 25 knots.
On Saturday, the United States confirmed that a balloon was observed transiting Central and South America, and said that it was another Chinese surveillance balloon.
“These balloons are all part of a P.R.C. fleet of balloons developed to conduct surveillance operations, which have also violated the sovereignty of other countries,” Brig. Gen. Patrick S. Ryder, the Pentagon’s press secretary, told reporters.
Dig deeper into the moment.
Special offer: Subscribe for $1 a week for the first year.
The Colombian Air Force said it had tracked the object until it left the country’s airspace, adding that officials determined it did not pose a threat to national security and were investigating its origin.
On Sunday, the Venezuelan Ministry of Foreign Affairs, referring to the U.S. decision to shoot down the balloon, said the government “rejects the attack by the United States against a civilian unmanned aircraft of Chinese origin.” In a statement, the Venezuelan government said, “Once again, the United States resorts to the use of force, instead of treating this situation with the seriousness and responsibility that the case deserves.”
President Nicolás Maduro of Venezuela has recently proposed establishing a new international bloc of Latin American and Caribbean countries and referred to Presidents Xi Jinping of China and Vladimir V. Putin of Russia as each being an “older brother.”
Editors’ Picks
A Secret for Falling Asleep So Good It’s a British National Treasure
When a Visit to the Museum Becomes an Ethical Dilemma
Is That the Enticing Smell of Spaghetti Cooking? Nope, It’s a Candle.
Ms. Mao said that the Chinese government had informed relevant officials in other nations about the balloon flying over Latin America and that “they have expressed their understanding.”
The United States, Ms. Mao said, had “a clear overreaction.”
Elda Cantú contributed reporting from Mexico City.
Emma Bubola is a reporter based in London. """

In [73]:
story_2 = """Balloon over Latin America belongs to China, Beijing says
A balloon spotted over the skies of Latin America belongs to China and was used for flight tests, a Chinese foreign ministry spokesperson said Monday in response to CNN’s queries.
This is the first time Beijing has admitted the balloon spotted over two Latin American countries belongs to China.
The balloon “seriously deviated” from its planned course and entered the skies over Latin America and the Caribbean “by mistake” due to weather conditions and limited control ability over the craft, Chinese foreign ministry spokesperson Mao Ning said in a regular press briefing on Monday.
A jet flies by a suspected Chinese spy balloon after shooting it down off the coast in Surfside Beach, South Carolina, U.S. February 4, 2023.  REUTERS/Randall Hill
Why the Chinese balloon crisis could be a defining moment in the new Cold War
This is the second Chinese balloon Beijing claims has drifted off course due to the weather after the US military on Saturday shot down a suspected Chinese spy balloon that flew over the continental US for days.
“China is a responsible country. We have always strictly abided by international law. We have informed all relevant parties and appropriately handled the situation, which did not pose any threats to any countries,” Mao said, adding that all parties “expressed their understanding.”
The Colombian Air Force and Costa Rica’s Civil Aviation Authority both confirmed that a white observation balloon similar to the one spotted over the US was tracked in their airspace last week, though they did not attribute the vessel to China.
"""

In [74]:
story_3 = """Second balloon over Latin America is ours - China
The Chinese government has admitted a balloon spotted over Latin America on Friday is from China - but claimed it is intended for civilian use.
Foreign ministry spokesperson Mao Ning said the aircraft had deviated from its route, having been blown off course.
A similar balloon was shot down in US airspace by military jets on Saturday amid allegations that it was being used for surveillance.
China has denied accusations of spying, saying it was monitoring the weather.
The incident has led to a diplomatic row between Washington and Beijing.
On Friday - before fighter jets brought down the balloon at the weekend - US military officials said a second Chinese balloon had been spotted over Latin America.
On Monday, China admitted an aircraft had "accidentally entered Latin American and Caribbean airspace".
Ms Mao told reporters the second balloon had "deviated greatly" from its intended route, citing the aircraft's "limited manoeuvrability" and the weather conditions.
"The unmanned airship in question that came from China is of a civilian nature and used for flight tests," she added.
"China is a responsible country and has always strictly abided by international law in order to inform and properly deal with all parties concerned, without posing any threat to any country."
At the weekend, Colombia's air force said an object with "characteristics similar to those of a balloon" had been detected on 3 February in the country's airspace at above 55,000ft.
Colombia said it had followed the object until it left the airspace, adding that it did not represent a threat to national security.
High-altitude spying marks new low for US-China ties
Was China balloon blown off course?
Why use a spy balloon instead of satellites?
Meanwhile, work by US Navy divers continues to recover the wreckage of the surveillance balloon that was shot down off the coast of South Carolina on Saturday.
US President Joe Biden first approved the plan to bring down the balloon on Wednesday, but decided to wait until it was over water so as not to put people on the ground at risk.
The US believes the balloon was being used to monitor sensitive military sites.
Adm Mike Mullen, former chair of the US Joint Chiefs of Staff, rejected China's suggestion it might have blown off course, saying it was manoeuvrable because "it has propellers on it".
"This was not an accident. This was deliberate. It was intelligence," he added.
Relations between China and the US have been strained by the incident, with the Pentagon calling it an "unacceptable violation" of its sovereignty. A planned trip by Secretary of State Antony Blinken to China was cancelled as a result of the row.
China has lodged a formal complaint with the US embassy in Beijing over the incident.
"""

In [123]:
s1 = get_summary(story_1, 8)
full_sum = s1
s1

'Give this article A Chinese balloon floating off the coast of South Carolina before being shot down by an American fighter jet on Saturday. “Affected by the weather and with limited self-steering capability, the airship deviated far from its planned course and entered into the airspace of Latin America and the Caribbean,” a spokeswoman for China’s foreign ministry, Mao Ning, said at a news conference on Monday. On Friday morning, before the United States shot down a Chinese spy balloon that had spent the last week traversing the country, the Colombian National Air Defense System detected an object that had entered the northern part of the country’s airspace, the Colombian Air Force said in a statement. Officials determined that the object had “characteristics similar to those of a balloon,” and that it was flying at an altitude of over 55,000 feet and moving at an average speed of 25 knots. fleet of balloons developed to conduct surveillance operations, which have also violated the so

In [124]:
s2 = get_summary(story_2, 8)
full_sum += s2
s2

'Balloon over Latin America belongs to China, Beijing says A balloon spotted over the skies of Latin America belongs to China and was used for flight tests, a Chinese foreign ministry spokesperson said Monday in response to CNN’s queries. This is the first time Beijing has admitted the balloon spotted over two Latin American countries belongs to China. The balloon “seriously deviated” from its planned course and entered the skies over Latin America and the Caribbean “by mistake” due to weather conditions and limited control ability over the craft, Chinese foreign ministry spokesperson Mao Ning said in a regular press briefing on Monday. A jet flies by a suspected Chinese spy balloon after shooting it down off the coast in Surfside Beach, South Carolina, U.S. February 4, 2023. REUTERS/Randall Hill Why the Chinese balloon crisis could be a defining moment in the new Cold War This is the second Chinese balloon Beijing claims has drifted off course due to the weather after the US military 

In [125]:
s3 = get_summary(story_3, 8)
full_sum += s3
print(s3)

A similar balloon was shot down in US airspace by military jets on Saturday amid allegations that it was being used for surveillance. On Monday, China admitted an aircraft had "accidentally entered Latin American and Caribbean airspace". Ms Mao told reporters the second balloon had "deviated greatly" from its intended route, citing the aircraft's "limited manoeuvrability" and the weather conditions. At the weekend, Colombia's air force said an object with "characteristics similar to those of a balloon" had been detected on 3 February in the country's airspace at above 55,000ft. Meanwhile, work by US Navy divers continues to recover the wreckage of the surveillance balloon that was shot down off the coast of South Carolina on Saturday. US President Joe Biden first approved the plan to bring down the balloon on Wednesday, but decided to wait until it was over water so as not to put people on the ground at risk. Adm Mike Mullen, former chair of the US Joint Chiefs of Staff, rejected China

In [126]:
full_sum_comp = get_summary(full_sum, 8)
print(full_sum_comp)

“Affected by the weather and with limited self-steering capability, the airship deviated far from its planned course and entered into the airspace of Latin America and the Caribbean,” a spokeswoman for China’s foreign ministry, Mao Ning, said at a news conference on Monday. The Colombian Air Force said it had tracked the object until it left the country’s airspace, adding that officials determined it did not pose a threat to national security and were investigating its origin. On Sunday, the Venezuelan Ministry of Foreign Affairs, referring to the U.S. decision to shoot down the balloon, said the government “rejects the attack by the United States against a civilian unmanned aircraft of Chinese origin.” In a statement, the Venezuelan government said, “Once again, the United States resorts to the use of force, instead of treating this situation with the seriousness and responsibility that the case deserves.” President Nicolás Maduro of Venezuela has recently proposed establishing a new 

In [130]:
full_sum_ents = text_to_ents(full_sum)
full_sum_ents

[('the colombian national air defense system', 'ORG'),
 ('nicolás maduro', 'PERSON'),
 ('civil aviation authority', 'ORG'),
 ('caribbean', 'LOC'),
 ('the united states', 'GPE'),
 ('mao', 'PERSON'),
 ('wednesday', 'DATE'),
 ('above 55,000', 'CARDINAL'),
 ('latin america', 'LOC'),
 ('over 55,000 feet', 'QUANTITY'),
 ('beijing', 'GPE'),
 ('chinese', 'NORP'),
 ('mao ning', 'PERSON'),
 ('british', 'NORP'),
 ('us', 'GPE'),
 ('saturday', 'DATE'),
 ('the weekend', 'DATE'),
 ('first', 'ORDINAL'),
 ('monday', 'DATE'),
 ('the museum becomes an ethical dilemma is that the enticing smell', 'ORG'),
 ('american', 'NORP'),
 ('reuters', 'ORG'),
 ('cold war', 'EVENT'),
 ('china', 'GPE'),
 ('south carolina', 'GPE'),
 ('u.s.', 'GPE'),
 ('two', 'CARDINAL'),
 ('joe biden', 'PERSON'),
 ('adm mike mullen', 'PERSON'),
 ('state', 'ORG'),
 ('costa rica’s', 'GPE'),
 ('russia', 'GPE'),
 ('days', 'DATE'),
 ('brig', 'PERSON'),
 ('the colombian air force', 'ORG'),
 ('february 4, 2023', 'DATE'),
 ('the us joint chiefs

In [159]:
new_story_summary_from_sums = generate_story([
    'https://www.nytimes.com/2023/02/06/world/americas/chinese-balloon-latin-america-colombia.html',
    'https://www.cnn.com/2023/02/06/asia/china-balloon-latin-america-intl/index.html',
    'https://www.bbc.com/news/world-64537098'
], full_sum_ents, full_sum_comp)
new_story_summary_from_sums

{
  "completion_tokens": 693,
  "prompt_tokens": 1042,
  "total_tokens": 1735
}


'\nIt had all started the Wednesday morning, when military personnel from the Colombian National Air Defense System first tracked a white observation balloon soaring through the skies at a height of over 55,000 feet. The Colombian authorities alerted the Civil Aviation Authority in neighboring Costa Rica, who were quick to take on the responsibility of monitoring the air-borne craft and reporting its trajectory when it came within a few miles of their own airspace. \n\nFor the better part of the week, the balloon’s journey remained largely unrestricted and unquestioned. That was until last Friday morning, when the US Joint Chiefs of Staff took note and identified the mysterious vessel as a Chinese craft. There had been simmering tensions between China and the United States over both economic and political issues that had reached another boiling point due to the US’s decision to enforce sanctions on Chinese companies in the lead up to the next Cold War. \n\nChinese foreign ministry spok

In [160]:
print(new_story_summary_from_sums)


It had all started the Wednesday morning, when military personnel from the Colombian National Air Defense System first tracked a white observation balloon soaring through the skies at a height of over 55,000 feet. The Colombian authorities alerted the Civil Aviation Authority in neighboring Costa Rica, who were quick to take on the responsibility of monitoring the air-borne craft and reporting its trajectory when it came within a few miles of their own airspace. 

For the better part of the week, the balloon’s journey remained largely unrestricted and unquestioned. That was until last Friday morning, when the US Joint Chiefs of Staff took note and identified the mysterious vessel as a Chinese craft. There had been simmering tensions between China and the United States over both economic and political issues that had reached another boiling point due to the US’s decision to enforce sanctions on Chinese companies in the lead up to the next Cold War. 

Chinese foreign ministry spokespers

In [161]:
new_story_from_sums_stacked = generate_story([
    'https://www.nytimes.com/2023/02/06/world/americas/chinese-balloon-latin-america-colombia.html',
    'https://www.cnn.com/2023/02/06/asia/china-balloon-latin-america-intl/index.html',
    'https://www.bbc.com/news/world-64537098'
], full_sum_ents, full_sum)
new_story_from_sums_stacked

{
  "completion_tokens": 641,
  "prompt_tokens": 1552,
  "total_tokens": 2193
}


'\nOn Saturday, the Colombian National Air Defense System detected a mysterious object entering the northern part of its airspace. The object had characteristics similar to a balloon and was moving at an altitude of over 55,000 feet and an average speed of 25 knots. Colombia immediately contacted its neighbor countries of Latin America and the Caribbean to notify them of the object, prompting the United States to take action and shoot down the suspected Chinese spy balloon.\n\nThe incident did not go unnoticed by Nicolás Maduro, the President of Venezuela, who quickly proposed a new international bloc of Latin American and Caribbean countries with Xi Jinping of China and Vladimir V. Putin of Russia as "older brothers". A spokeswoman for China\'s foreign ministry, Mao Ning, chimed in as well, telling reporters at a news conference that the Chinese government had informed relevant officials about the balloon before it entered Latin American airspace, and that the U.S. had a clear “overre

In [163]:
wandb.log({'predictions': prediction_table})
wandb.finish()

In [162]:
print(new_story_from_sums_stacked)


On Saturday, the Colombian National Air Defense System detected a mysterious object entering the northern part of its airspace. The object had characteristics similar to a balloon and was moving at an altitude of over 55,000 feet and an average speed of 25 knots. Colombia immediately contacted its neighbor countries of Latin America and the Caribbean to notify them of the object, prompting the United States to take action and shoot down the suspected Chinese spy balloon.

The incident did not go unnoticed by Nicolás Maduro, the President of Venezuela, who quickly proposed a new international bloc of Latin American and Caribbean countries with Xi Jinping of China and Vladimir V. Putin of Russia as "older brothers". A spokeswoman for China's foreign ministry, Mao Ning, chimed in as well, telling reporters at a news conference that the Chinese government had informed relevant officials about the balloon before it entered Latin American airspace, and that the U.S. had a clear “overreactio

In [98]:
bass2 = """
Kamala Harris to swear in Karen Bass as first female mayor of Los Angeles
Vice President Kamala Harris will swear in Karen Bass as the 43rd mayor of Los Angeles in an inauguration ceremony on Sunday in a meeting of two Democratic women who have broken barriers in politics.
Bass will make history as the first female mayor of Los Angeles, also making her the first woman of color to hold the job. Harris, just the second-ever Black female US senator, made history when she became the first female, first Black and first South Asian person to serve as vice president.
Bass, who will succeed term-limited Mayor Eric Garcetti, will begin her mayoral term on December 12. When she takes office, the four largest cities in the US will all have Black mayors – that includes Eric Adams of New York City, Lori Lightfoot of Chicago and Sylvester Turner of Houston.
Bass, who was elected to the California state Assembly in 2004, made history some four years later as the first Black woman to serve as speaker of any state legislature.
Bass alluded to her history of firsts shortly after winning her race when she told reporters that the accomplishment of becoming the first female mayor of Los Angeles was still “sinking in.”
“When you’re in a position like this – and I was in a similar position when I was sworn in as speaker – it means that you have extra responsibility. You always have to make sure that you maintain excellence in every step of the way,” Bass said. “The path that you go is laying the foundation for those that come behind you.”
After her victory, Bass promised to solve the homelessness crisis in the nation’s second-largest city.
“She is holding her inaugural ceremony on Sunday to make it easier for Angelenos to attend and to ensure her first day as mayor is dedicated to bringing unhoused Angelenos inside and making our city safer and more livable for all,” the mayor-elect’s office said in a statement announcing the ceremony.
The six-term congresswoman, who currently represents South and West Los Angeles, defeated real estate developer Rick Caruso in the general election last month. Caruso had spent more than $104 million – outspending his opponent by more than 11-to-1.
During her campaign, Bass leaned into her experience bringing together Black and Latino community organizers in South Los Angeles in the early 1990s to address the root causes of crime and the crack epidemic. She was able to put together a winning coalition of Black voters in South Los Angeles and White progressives on the city’s west side to help overcome Caruso’s spending.
The mayor-elect began her career as a physician assistant in the emergency room in Los Angeles County.
Joe Biden had vetted Bass, the then-chair of the Congressional Black Caucus, to be his running mate in 2020 as she helped lead the negotiations on legislation to create greater police accountability following the police killing of George Floyd in Minneapolis.
"""

In [99]:
b2 = get_summary(bass2, 8)
print(b2)

Kamala Harris to swear in Karen Bass as first female mayor of Los Angeles Vice President Kamala Harris will swear in Karen Bass as the 43rd mayor of Los Angeles in an inauguration ceremony on Sunday in a meeting of two Democratic women who have broken barriers in politics. Bass alluded to her history of firsts shortly after winning her race when she told reporters that the accomplishment of becoming the first female mayor of Los Angeles was still “sinking in.” “When you’re in a position like this – and I was in a similar position when I was sworn in as speaker – it means that you have extra responsibility. “The path that you go is laying the foundation for those that come behind you.” After her victory, Bass promised to solve the homelessness crisis in the nation’s second-largest city. “She is holding her inaugural ceremony on Sunday to make it easier for Angelenos to attend and to ensure her first day as mayor is dedicated to bringing unhoused Angelenos inside and making our city safe

In [100]:
bass3 = """
Karen Bass sworn in as LA's 43rd mayor in historic inauguration
Karen Bass has been sworn in as the 43rd Mayor of Los Angeles in a historic inauguration at the Microsoft Theater. 
The ceremony got underway at 1 p.m. with early arrival encouraged and masks required for attendees. The ceremony was initially scheduled to take place outside of City Hall, but due to the rain, was moved indoors. 
Instead of being sworn in on City Hall's steps as tradition would have it, the city adapted to the powerful storm moving through the area, posting the letters "LA" in the mold of the LAX sign to her right and a picture of the Spring Street City Hall steps behind the stage.
Ready to get down to business, Bass said she would spend her first day in office declaring a state of emergency on the homeless issue plaguing the city. 
"I will start my first day as mayor at the city's emergency operations centers, where my first act as mayor will be to declare a state of emergency on homelessness," Bass said, noting that she will "recognize the severity of our crisis and break new ground to maximize our ability to urgently move people inside, and do so for good."
She plans to house 17,000 people in her first year in office, hoping to "welcome housing to every neighborhood."
"We know our mission: We must build housing in every neighborhood," she said. "We cannot continue to overcrowd neighborhoods that are already overcrowded."
Bass also touched on another pressing issue facing her in her first day as mayor. 
"Of course, we must stop crimes in progress and hold people accountable," Bass said. "Some neighborhoods have asked for additional officers, and we will deliver. But what neighborhoods are asking for and what they need is as diverse as our city."
Dozens of prominent politicians were in attendance for the inauguration, including California State Senate President pro-tempore Toni Atkins, Los Angeles County Democratic Party Chairman Mark Gonzalez and Governor Gavin Newsom.
"Making history with each of you today is a monumental moment in my life and for Los Angeles," Bass said during her speech.
Additionally, Stevie Wonder sang for the crowd, performing "Living for the City" and "Keeping Our Love Alive." Chloe Bailey and Los Cafeteras also performed. 
Bass, 69, was a social worker and community activist before representing Los Angeles in the state Assembly for six years, where she also served as the first African American speaker. 
In 2010, she was elected to Congress and served Los Angeles before coming back home and becoming the first Black female mayor of Los Angeles. 
"I believe that times of inflection require reflection — I believe, it's time for Angelenos to remind ourselves where we come from and who we are," she said.
She takes over in a time of turmoil, as the Los Angeles City Council struggles to conduct business through a racially-charged scandal involving several prominent members, but also at a time of hope, as the council will hold more women than ever before after the five newest members swore in over the weekend. 
Council President Paul Krekorian spoke at the event, predicting that Bass is just the leader the city of Los Angeles needs to bring people together. 
"This is a time of unprecedented challenges in our city, but today as I look out at this audience and see the people with us, I know that this is also a time of unprecedented opportunity," Krekorian said.
Bass will replace fellow Democrat Eric Garcetti, who came to the end of two mayoral terms on Saturday. He now awaits further plans pending his nomination to become U.S. ambassador to India, which has hit a snag due to opposition from some on Senate, concerned with sexual misconduct allegations against a former Garcetti staff member.
Bass was named mayor-elect in November after a close race with her competitor Rick Caruso, the billionaire developer who spent more than $100 million on his campaign. 
"""

In [101]:
b3 = get_summary(bass3, 8)
print(b3)

The ceremony was initially scheduled to take place outside of City Hall, but due to the rain, was moved indoors. Ready to get down to business, Bass said she would spend her first day in office declaring a state of emergency on the homeless issue plaguing the city. "I will start my first day as mayor at the city's emergency operations centers, where my first act as mayor will be to declare a state of emergency on homelessness," Bass said, noting that she will "recognize the severity of our crisis and break new ground to maximize our ability to urgently move people inside, and do so for good." Dozens of prominent politicians were in attendance for the inauguration, including California State Senate President pro-tempore Toni Atkins, Los Angeles County Democratic Party Chairman Mark Gonzalez and Governor Gavin Newsom. Bass, 69, was a social worker and community activist before representing Los Angeles in the state Assembly for six years, where she also served as the first African America

In [103]:
bass1 = url_to_string('https://socalnews.com/la-story-3.html')
b1 = get_summary(bass1, 8)
b1

"The emergency declaration will ``recognize the severity of our crisis and break new ground to maximize our ability to urgently move people inside, and do so for good. Sunday's ceremony was initially scheduled to take place outside City Hall, but rain in the forecast led to a venue shift indoors. Surprise performers at the inauguration included Stevie Wonder, who sang ``Keep Our Love Alive'' and ``Living for the City,'' and Chloe Bailey and Las Cafeteras. Amanda Gorman, the first National Youth Poet Laureate, who also delivered a poem at President Joe Biden's inauguration, said in her reading Sunday that ``the time of never before is officially past.'' Bass' father fled the Jim Crow south during the great African American migration after World War II and found work as a letter carrier. ``When I think about the dreams of working people today, I reflect on the fact that my mother and father were able to buy a home in Los Angeles for their family of six with one paycheck,'' Bass said. Bas

In [107]:
bass_sums_stacked = b1 + b2 + b3
print(bass_sums_stacked)

The emergency declaration will ``recognize the severity of our crisis and break new ground to maximize our ability to urgently move people inside, and do so for good. Sunday's ceremony was initially scheduled to take place outside City Hall, but rain in the forecast led to a venue shift indoors. Surprise performers at the inauguration included Stevie Wonder, who sang ``Keep Our Love Alive'' and ``Living for the City,'' and Chloe Bailey and Las Cafeteras. Amanda Gorman, the first National Youth Poet Laureate, who also delivered a poem at President Joe Biden's inauguration, said in her reading Sunday that ``the time of never before is officially past.'' Bass' father fled the Jim Crow south during the great African American migration after World War II and found work as a letter carrier. ``When I think about the dreams of working people today, I reflect on the fact that my mother and father were able to buy a home in Los Angeles for their family of six with one paycheck,'' Bass said. Bass

In [106]:
bass_sum_from_sums = get_summary(b1 + b2 + b3, 8)
bass_sum_from_sums

"Amanda Gorman, the first National Youth Poet Laureate, who also delivered a poem at President Joe Biden's inauguration, said in her reading Sunday that ``the time of never before is officially past.'' Bass' father fled the Jim Crow south during the great African American migration after World War II and found work as a letter carrier. Bass' organizing career began in 1990 when she founded Community Coalition, a South Los Angeles social justice group in response to the crack cocaine crisis. During her campaign, Bass leaned into her experience bringing together Black and Latino community organizers in South Los Angeles in the early 1990s to address the root causes of crime and the crack epidemic. The mayor-elect began her career as a physician assistant in the emergency room in Los Angeles County.The ceremony was initially scheduled to take place outside of City Hall, but due to the rain, was moved indoors. Dozens of prominent politicians were in attendance for the inauguration, includi

In [168]:
bass_from_stacked = generate_story([
    'https://socalnews.com/la-story-3.html',
    'https://www.cnn.com/2022/12/05/politics/karen-bass-kamala-harris-inauguration/index.html',
    'https://www.cbsnews.com/losangeles/news/karen-bass-sworn-in-as-43rd-mayor-of-los-angeles-inauguration/'
], entities, bass_sums_stacked)
bass_from_stacked

{
  "completion_tokens": 740,
  "prompt_tokens": 1650,
  "total_tokens": 2390
}


'\nIt was just days before Karen Bass\' mayoral inauguration ceremony when the Los Angeles City council voted to make her the first female mayor of the City of Los Angeles. The 69-year-old Bass had first broken into politics nearly a decade before as a member of the California State legislature, representing the Los Angeles County. On Sunday, January 14, 2022, history would be made as she became the 43rd mayor of Los Angeles.\n\nThe ceremony was scheduled to take place at City Hall, but due to bad weather it was shifted indoors to the Microsoft Theater. A confirmed list of guests included Vice President Kamala Harris who swore in Bass as mayor, California State Senate President pro-Tempore Toni Atkins, Los Angeles County Democratic Party Chair Mark Gonzalez, and Governor Gavin Newsom. Bass was not alone in making history, however– Amanda Gorman, first National Youth Poet Laureate, made a surprise appearance and said, “The time of never before is officially past.”\n\nAs her story is int

In [169]:
print(bass_from_stacked)


It was just days before Karen Bass' mayoral inauguration ceremony when the Los Angeles City council voted to make her the first female mayor of the City of Los Angeles. The 69-year-old Bass had first broken into politics nearly a decade before as a member of the California State legislature, representing the Los Angeles County. On Sunday, January 14, 2022, history would be made as she became the 43rd mayor of Los Angeles.

The ceremony was scheduled to take place at City Hall, but due to bad weather it was shifted indoors to the Microsoft Theater. A confirmed list of guests included Vice President Kamala Harris who swore in Bass as mayor, California State Senate President pro-Tempore Toni Atkins, Los Angeles County Democratic Party Chair Mark Gonzalez, and Governor Gavin Newsom. Bass was not alone in making history, however– Amanda Gorman, first National Youth Poet Laureate, made a surprise appearance and said, “The time of never before is officially past.”

As her story is intertwine

In [170]:
bass_from_sums = generate_story([
    'https://socalnews.com/la-story-3.html',
    'https://www.cnn.com/2022/12/05/politics/karen-bass-kamala-harris-inauguration/index.html',
    'https://www.cbsnews.com/losangeles/news/karen-bass-sworn-in-as-43rd-mayor-of-los-angeles-inauguration/'
], entities, bass_sum_from_sums)
bass_from_sums

{
  "completion_tokens": 881,
  "prompt_tokens": 1020,
  "total_tokens": 1901
}


'\n\nIt was a historic day for Los Angeles on Monday as Sharon Bass, 69, was sworn in as the city’s first female Black mayor. The inauguration, which was originally going to be held outside City Hall, was moved indoors due to rain but that didn’t dampen the festive spirit of the day. A large crowd gathered to witness the special occasion, including prominent politicians such as California State Senate President pro-tempore Toni Atkins, Los Angeles County Democratic Party Chairman Mark Gonzalez, Governor Gavin Newsom, and Kamala Harris.\n\nMayor-elect Bass began her career in 1990 as a physician assistant in the emergency room at a Los Angeles County hospital. But it was her founding of the South Los Angeles social justice organization, Community Coalition, in response to the crack cocaine crisis that really had an impact. This experience of bringing together Black and Latino community organizers to tackle the root causes of crime and drug addiction in the city established her as a lead

In [171]:
print(bass_from_sums)



It was a historic day for Los Angeles on Monday as Sharon Bass, 69, was sworn in as the city’s first female Black mayor. The inauguration, which was originally going to be held outside City Hall, was moved indoors due to rain but that didn’t dampen the festive spirit of the day. A large crowd gathered to witness the special occasion, including prominent politicians such as California State Senate President pro-tempore Toni Atkins, Los Angeles County Democratic Party Chairman Mark Gonzalez, Governor Gavin Newsom, and Kamala Harris.

Mayor-elect Bass began her career in 1990 as a physician assistant in the emergency room at a Los Angeles County hospital. But it was her founding of the South Los Angeles social justice organization, Community Coalition, in response to the crack cocaine crisis that really had an impact. This experience of bringing together Black and Latino community organizers to tackle the root causes of crime and drug addiction in the city established her as a leader in

In [172]:
wandb.log({'predictions': prediction_table})
wandb.finish()